##**Project Idea:** Simple Waste Material Type Classifier

**Why this idea:** Helps automate waste sorting, improving recycling efficiency

**Labels:**
- Plastic Waste
- Glass Waste
- Paper Waste
- Cardboard Waste
- Metal Waste
- Organic Waste
- Textiles Waste
- Electronics Waste
- Batteries Waste
- Wood Waste
- Rubber Waste
- Food Waste
- Mixed Waste

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
bs = 32
version = 0

In [ ]:
pip install -Uqq fastai

In [ ]:
from fastai import *
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/MasterCourse/Capstone_2'

/content/drive/MyDrive/MasterCourse/Capstone_2


In [ ]:
data_path = 'waste_data'
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.1, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
dblock = dblock.new(item_tfms=RandomResizedCrop(224,min_scale=0.5),batch_tfms=aug_transforms())
dls=dblock.dataloaders(data_path, bs=bs)
torch.save(dls,f'waste_classifier_v{version}.pkl')

In [ ]:
dls.train.vocab

['Batteries Waste', 'Cardboard Waste', 'Electronics Waste', 'Glass Waste', 'Metal Waste', 'Organic Waste', 'Paper Waste', 'Plastic Waste', 'Rubber Waste', 'Textiles Waste', 'Wood Waste']

In [ ]:
model_path = 'models'
device = torch.device('cuda')

In [ ]:
model = vision_learner(dls, resnet34, metrics=[error_rate, accuracy])


In [ ]:
model.fine_tune(5)

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,2.536180,1.107987,0.336842,0.663158,03:42


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,1.141856,0.631221,0.168421,0.831579,05:25
1,0.912075,0.784175,0.231579,0.768421,05:46
2,0.698308,0.588047,0.178947,0.821053,05:39
3,0.536523,0.625293,0.157895,0.842105,05:48
4,0.430797,0.620869,0.157895,0.842105,05:46


In [ ]:
model.save(f'waste_classifier_v{version}')


Path('models/waste_classifier_v0.pth')

In [ ]:
cleaner = ImageClassifierCleaner(model)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unline()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]),f"{data_path}/{cat}")

In [ ]:
model.export(f"{model_path}/waste_classifier_v{version}.pkl")